In [20]:
import os
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import wandb


In [24]:
def train():
    run = wandb.init(project="rf-window-classification", job_type="training")
    config = run.config

    # 1) Cargar CSV
    feat_file = f"./features_csvs/features_fixed_{config.window_size_seconds}s.csv"
    if not os.path.exists(feat_file):
        raise FileNotFoundError(f"Archivo no encontrado: {feat_file}")

    df = pd.read_csv(feat_file)

    # 2) Eliminar columnas de metadata si están presentes
    candidate_cols = ["video_id", "segment", "participant", "window_i", "window_s", "run_count", "modality"]
    drop_cols = [c for c in candidate_cols if c in df.columns]
    df = df.drop(columns=drop_cols)

    # 3) Separar X e y
    df["label"] = df["label"].astype(int)
    feature_cols = df.select_dtypes(include=[np.number]).columns.drop("label")
    X = df[feature_cols].values
    y = df["label"].values

    # 4) Escalado opcional
    if config.scaler_type == "standard":
        scaler = StandardScaler()
    elif config.scaler_type == "minmax":
        scaler = MinMaxScaler()
    else:
        scaler = None

    if scaler is not None:
        X = scaler.fit_transform(X)
    wandb.log({"preprocessing/scaler_type": config.scaler_type})

    # 5) Split en entrenamiento y validación
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=config.random_state
    )

    # 6) Entrenar modelo Random Forest
    clf = RandomForestClassifier(
        n_estimators=config.n_estimators,
        max_depth=config.max_depth,
        class_weight=config.class_weight,
        random_state=config.random_state
    )
    clf.fit(X_train, y_train)

    # 7) Evaluación
    y_pred = clf.predict(X_val)
    f1 = f1_score(y_val, y_pred, average="macro")
    wandb.log({"val/f1_macro": f1})
    print(f"✅ F1-macro = {f1:.3f}")

    # 8) Classification report
    report = classification_report(y_val, y_pred, output_dict=True, zero_division=0)
    for label, metrics in report.items():
        if isinstance(metrics, dict):
            for metric_name, value in metrics.items():
                wandb.log({f"{label}/{metric_name}": value})
        else:
            wandb.log({label: metrics})

    run.finish()


In [27]:
sweep_config = {
    "method": "bayes",
    "metric": {
        "name": "val/f1_macro",
        "goal": "maximize"
    },
    "parameters": {
        "window_size_seconds": {
            "values": [1, 5, 10]
        },
        "n_estimators": {
            "values": [50, 100, 200]
        },
        "max_depth": {
            "values": [5, 10, 20]
        },
        "random_state": {
            "value": 42
        },
        "class_weight": {
            "values": ["balanced", "balanced_subsample"]
        },
        "scaler_type": {
            "values": ["standard"]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="rf-sequence-classification")
print("Sweep ID:", sweep_id)


Create sweep with ID: 0hueicyq
Sweep URL: https://wandb.ai/knezevicoluka-tu-delft/rf-sequence-classification/sweeps/0hueicyq
Sweep ID: 0hueicyq


In [28]:
# ─── Cell 4: Launch agents directly from the notebook ─────────────────────────

# You can spin up multiple agents (in parallel or sequentially) by running this cell multiple times.
# Each agent will pull one new config from the sweep and run train() under that config.

wandb.agent(sweep_id, function=train, count=10)

# - `count=10` means “run 10 different trials” (or until the sweep ends).
# - Omit `count` if you want to keep going until you manually stop it or exhausting the search space.


wandb: Agent Starting Run: jbru3c0w with config:
wandb: 	class_weight: balanced_subsample
wandb: 	max_depth: 20
wandb: 	n_estimators: 100
wandb: 	random_state: 42
wandb: 	scaler_type: standard
wandb: 	window_size_seconds: 10


✅ F1-macro = 0.519


0/f1-score,▁
0/precision,▁
0/recall,▁
0/support,▁
1/f1-score,▁
1/precision,▁
1/recall,▁
1/support,▁
accuracy,▁
macro avg/f1-score,▁
macro avg/precision,▁


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: f4karbzc with config:
wandb: 	class_weight: balanced
wandb: 	max_depth: 10
wandb: 	n_estimators: 50
wandb: 	random_state: 42
wandb: 	scaler_type: standard
wandb: 	window_size_seconds: 10


✅ F1-macro = 0.498


...

✅ F1-macro = 0.520


0/f1-score,▁
0/precision,▁
0/recall,▁
0/support,▁
1/f1-score,▁
1/precision,▁
1/recall,▁
1/support,▁
accuracy,▁
macro avg/f1-score,▁
macro avg/precision,▁


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: b8qlwsy5 with config:
wandb: 	class_weight: balanced_subsample
wandb: 	max_depth: 20
wandb: 	n_estimators: 50
wandb: 	random_state: 42
wandb: 	scaler_type: standard
wandb: 	window_size_seconds: 10


✅ F1-macro = 0.520


0/f1-score,▁
0/precision,▁
0/recall,▁
0/support,▁
1/f1-score,▁
1/precision,▁
1/recall,▁
1/support,▁
accuracy,▁
macro avg/f1-score,▁
macro avg/precision,▁
